In [ ]:
!pip install -q tensorflow
!pip install -q keras

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import regex as re


In [ ]:
def file_to_sentence_list(file_path):
	with open(file_path, 'r') as file:
		text = file.read()

	sentences = [sentence.strip() for sentence in re.split(
		r'(?<=[.!?])\s+', text) if sentence.strip()]

	return sentences

file_path = 'pizza.txt'
text_data = file_to_sentence_list(file_path)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in text_data:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(
	input_sequences, maxlen=max_sequence_len, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]


y = tf.keras.utils.to_categorical(y, num_classes=total_words)


In [ ]:

model = Sequential()
model.add(Embedding(total_words, 10,
					input_length=max_sequence_len-1))
model.add(LSTM(128))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy',
			optimizer='adam', metrics=['accuracy'])


In [ ]:

model.fit(X, y, epochs=1000, verbose=1)


Epoch 1/1000
51/51 [==============================] - 4s 83ms/step - loss: 5.7086 - accuracy: 0.0461
Epoch 2/1000
51/51 [==============================] - 3s 57ms/step - loss: 5.6731 - accuracy: 0.0504
Epoch 3/1000
51/51 [==============================] - 3s 53ms/step - loss: 5.6249 - accuracy: 0.0571
Epoch 4/1000
51/51 [==============================] - 3s 58ms/step - loss: 5.5743 - accuracy: 0.0577
Epoch 5/1000
51/51 [==============================] - 3s 69ms/step - loss: 5.5241 - accuracy: 0.0608
Epoch 6/1000
51/51 [==============================] - 4s 73ms/step - loss: 5.4728 - accuracy: 0.0596
Epoch 7/1000
51/51 [==============================] - 3s 54ms/step - loss: 5.4164 - accuracy: 0.0676
Epoch 8/1000
51/51 [==============================] - 3s 54ms/step - loss: 5.3463 - accuracy: 0.0731
Epoch 9/1000
51/51 [==============================] - 3s 55ms/step - loss: 5.2618 - accuracy: 0.0774
Epoch 10/1000
51/51 [==============================] - 4s 88ms/step - loss: 5.1701 - accura

In [ ]:

seed_text = "Pizza have different "
next_words = 5

for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences(
		[token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted_probs = model.predict(token_list)
	predicted_word = tokenizer.index_word[np.argmax(predicted_probs)]
	seed_text += " " + predicted_word

print("Next predicted words:", seed_text)


1/1 [==============================] - 0s 15ms/step
Next predicted words: Pizza have different  frango com catupiry is a
